In [1]:
_RIOT_API_KEY='RGAPI-01b62259-1df1-4a3c-b15c-88b29230dc98'
from riotwatcher import LolWatcher
import helper_functions as help
import constants
import pandas as pd
from datetime import datetime
import mysql.connector
from sqlalchemy import create_engine

lol_watcher = LolWatcher(_RIOT_API_KEY)


1. get puuid for a summoner name - Summonerv4/by-name/summonername
2. get list of match ids by puuid - Matchv5/ids
3. Check database for which match ids i don't have - find_to_slurp
4. Create a function that does the following
- get match by match id - Matchv5/matchids
    - Store in a DB
- get match timeline by match id
    - store in a DB
5. Create concurrent processing that will speed this up

# Match Timeline Work

In [4]:
match_timeline = help.slurp_match_timeline(lol_watcher, match_list[0], region='na1')
#match_timeline

# DB Create Table and Upload

In [5]:
match_timeline

,matchId,minute,participantId,totalGold,level,xp,minionsKilled,jungleMinionsKilled,totalDamageDoneToChampions,posX,posY
0,NA1_4721011388,0,1,0,3,2000,0,0,0,9513,9193
1,NA1_4721011388,0,2,0,3,2000,0,0,0,9488,8734
2,NA1_4721011388,0,3,0,3,2000,0,0,0,9146,8426
3,NA1_4721011388,0,4,0,3,2000,0,0,0,8687,8451
4,NA1_4721011388,0,5,0,3,2000,0,0,0,8379,8793
...,...,...,...,...,...,...,...,...,...,...,...
147,NA1_4721011388,18,4,12410,13,12500,0,0,17402,9012,8300
148,NA1_4721011388,18,5,23770,18,17000,2,0,42569,9146,8426
149,NA1_4721011388,18,6,23650,18,17000,1,0,38182,9180,8721
150,NA1_4721011388,18,7,23395,18,17043,5,0,62923,9012,8300


In [46]:
user = 'root'
password = 'iamgroot482'
port = '3306'
host = '127.0.0.1'
database = 'riot_api'
url="mysql+mysqlconnector://{0}:{1}@{2}:{3}/{4}".format(
            user, password, host, port, database
        )


engine = create_engine(url)
test_pull_df = pd.read_sql('SELECT * FROM lol_match_details', con=engine)
test_pull_df
#match_details.to_sql(con=engine, name='lol_match_details', if_exists='append', index = False)

,allInPings,assistMePings,assists,baitPings,baronKills,basicPings,bountyLevel,champExperience,champLevel,championId,...,teamName,item1Name,item2Name,item3Name,item4Name,item5Name,item6Name,summoner1Name,summoner2Name,queueName
0,0,0,2,0,0,0,0,12500,13,22,...,Red,Guardian's Hammer,Blade of The Ruined King,Wit's End,Titanic Hydra,None,Arcane Sweeper,Flee,Flash,Arena
1,0,0,3,0,0,0,0,12500,13,33,...,Red,Guardian's Amulet,"Jak'Sho, The Protean",Spirit Visage,Force of Nature,None,Arcane Sweeper,Flee,Flash,Arena
2,0,0,4,0,0,0,0,12500,13,74,...,Red,Zhonya's Hourglass,Luden's Tempest,Rabadon's Deathcap,Sorcerer's Shoes,None,Arcane Sweeper,Flee,Flash,Arena
3,0,0,1,0,0,0,0,12500,13,35,...,Red,Guardian's Orb,Liandry's Anguish,Rylai's Crystal Scepter,Lich Bane,None,Arcane Sweeper,Flee,Flash,Arena
4,0,0,10,0,0,0,0,17000,18,61,...,Blue,Lich Bane,Luden's Tempest,Seraph's Embrace,Rabadon's Deathcap,Night Harvester,Arcane Sweeper,Flee,Flash,Arena
5,0,0,8,0,0,0,1,17000,18,22,...,Blue,Bloodthirster,Wit's End,Immortal Shieldbow,Rapid Firecannon,Mercury's Treads,Arcane Sweeper,Flee,Flash,Arena
6,0,0,6,0,0,0,0,17043,18,74,...,Blue,Morellonomicon,Luden's Tempest,Rite Of Ruin,Demonic Embrace,Shadowflame,Arcane Sweeper,Flee,Flash,Arena
7,0,0,4,0,0,0,0,17043,18,55,...,Blue,Atma's Reckoning,Hextech Gunblade,Riftmaker,Rabadon's Deathcap,Rite Of Ruin,Arcane Sweeper,Flee,Flash,Arena


152

# Find To Slurp


In [5]:

user = 'root'
password = 'iamgroot482'
port = '3306'
host = '127.0.0.1'
database = 'riot_api'
url="mysql+mysqlconnector://{0}:{1}@{2}:{3}/{4}".format(
            user, password, host, port, database
        )
engine = create_engine(url)    
to_slurp = help.find_to_slurp('souporsecret', lol_watcher, 'lol_match_timeline', engine)


In [9]:
for matchid in to_slurp[:2]:
    match_timeline = help.slurp_match_timeline(lol_watcher, matchid, region='na1')
    match_timeline.to_sql(con=engine, name='lol_match_timeline', if_exists='append', index = False)

In [ ]:
## This is the mother function you run.
## summoner_name  (string) - Name of summoner to pull data for
## lol_watcher (object) - Authenticated lol_watcher object
## table_name (string) - Table we want to populate. either 'lol_match_timeline' or 'lol_match_details'
## db_engine (sqlAlchemy Engine Object)- Connection to mySQLDatabase from sqlAlchemy's create_engine.
##                                       Set database = 'riot_api' in parameters.
## region (string) - Region account is in. Defaults to NA
def collect_riot_api_data(summoner_name, lol_watcher, table_name, db_engine, region='na1'):
    # First We run find_to_slurp to avoid pulling data for matches we have already pulled
    to_slurp = help.find_to_slurp(summoner_name, lol_watcher, table_name, db_engine)

    # Next we use slurp_data to pull the data from Riot API
    # Set Up multiprocessing for this later
    for match_id in to_slurp[:10]:
        df_to_upload = help.slurp_data(lol_watcher, match_id, table_name, region='na1')
        df_to_upload.to_sql(con=db_engine, name=table_name, if_exists='append', index = False)
    print('Done')